In [2]:
from attack_predictor_libs.dataset.guide_dataset import GuideDataset
import polars as pl
from attack_predictor_libs.visualize.scenario_visualizer import visualize_scenario
from tqdm import tqdm

## 目的
GUIDEデータセットには大量のデータがあるが、使用困難なデータもあると思われる。  


In [3]:
dataset = GuideDataset()

In [4]:
df  = dataset.data.filter(pl.col("dataset_type") == 'train').filter(pl.col("IncidentGrade") == "TruePositive")

## 解析処理
仮説：攻撃は、Preparation, Intrusion, Compromiseのこの３つの順番に処理が進んでいる。  
全てのシナリオがPreparation, Intrusion, Compromise、この３つの順番に処理が進んでいるかを確認する。

In [94]:
ret_dict = {}
gdf = df.group_by("IncidentId")
for key, group_df in tqdm(gdf, total=df.n_unique("IncidentId")):
    
    tmpdf = group_df.with_columns(
        pl.max_horizontal(
            pl.col("is_preparation_alert").cast(int),
            pl.col("is_intrusion_alert").cast(int)*2,
            pl.col("is_compromise_alert").cast(int)*4
        ).alias("phase"),
    ).sort("Timestamp")

    min_value = tmpdf.select(
        pl.col("phase").diff().fill_null(0).alias("phase_diff")
    ).get_column("phase_diff").min()
    
    
    # keyはincident_id, min_valueはマイナスであれば、Preparation, Intrusino, Compromiseの順でアラートが来て以内ということを示す。
    ret_dict[key] = min_value
    
    # print(key)

  0%|          | 0/42701 [00:00<?, ?it/s]

100%|██████████| 42701/42701 [02:07<00:00, 336.05it/s]


In [95]:
incident_df = df.filter(pl.col("IncidentId") == 19927)
incident_df = incident_df.with_columns(
    pl.max_horizontal(
        pl.col("is_preparation_alert").cast(int),
        pl.col("is_intrusion_alert").cast(int)*2,
        pl.col("is_compromise_alert").cast(int)*4
    ).alias("phase")
    .diff().fill_null(0).alias("phase_diff")
)
incident_df.head()


Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,ActionGrouped,ActionGranular,EntityType,EvidenceRole,DeviceId,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,EmailClusterId,RegistryKey,RegistryValueName,RegistryValueData,ApplicationId,ApplicationName,OAuthApplicationId,ThreatFamily,FileName,FolderPath,ResourceIdName,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City,dataset_type,is_preparation_alert,is_intrusion_alert,is_compromise_alert,alert_bit_value,phase_diff
i64,i64,i64,i64,datetime[μs],i64,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,str,str,i64,i64,str,str,str,i64,i64,i64,str,bool,bool,bool,i64,i64
1675037249857,564,19927,22756,2024-06-13 08:27:05,376,65,"""Exfiltration""","""T1041""","""TruePositive""",null,null,"""File""","""Related""",98799,138268,360606,160396,441377,673934,425863,453297,153085,529644,null,1631,635,860,2251,3421,881,null,266788,92121,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",false,false,true,4,0
1675037249857,564,19927,22756,2024-06-13 08:27:05,376,65,"""Exfiltration""","""T1041""","""TruePositive""",null,null,"""File""","""Related""",98799,138268,360606,160396,441377,673934,425863,453297,153085,529644,null,1631,635,860,2251,3421,881,null,53711,92126,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",false,false,true,4,0
1675037249857,564,19927,22756,2024-06-13 08:27:05,376,65,"""Exfiltration""","""T1041""","""TruePositive""",null,null,"""File""","""Related""",98799,138268,360606,160396,441377,673934,425863,453297,153085,529644,null,1631,635,860,2251,3421,881,null,242617,92127,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",false,false,true,4,0
1675037249857,564,19927,22756,2024-06-13 08:27:05,376,65,"""Exfiltration""","""T1041""","""TruePositive""",null,null,"""File""","""Related""",98799,138268,360606,160396,441377,673934,425863,453297,153085,529644,null,1631,635,860,2251,3421,881,null,200713,96575,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",false,false,true,4,0
1675037249857,564,19927,22756,2024-06-13 08:27:05,376,65,"""Exfiltration""","""T1041""","""TruePositive""",null,null,"""File""","""Related""",98799,138268,360606,160396,441377,673934,425863,453297,153085,529644,null,1631,635,860,2251,3421,881,null,235148,96023,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",false,false,true,4,0


In [103]:
len(ret_dict)

42701

In [106]:
counter

907

In [105]:
counter = 0
for key in ret_dict.keys():
    if ret_dict[key] < 0:
        counter += 1
        print(key, ret_dict[key])
        
        # visualize_scenario(key, dataset)
# ret_dict

(18748,) -1
(265,) -1
(12195,) -1
(68165,) -1
(5773,) -1
(12847,) -1
(16003,) -1
(128729,) -1
(46931,) -1
(74459,) -1
(98182,) -1
(7342,) -1
(268,) -1
(40384,) -1
(795,) -1
(533,) -1
(209598,) -1
(146816,) -1
(137,) -1
(14687,) -2
(7345,) -3
(35141,) -1
(42602,) -1
(28448,) -1
(3939,) -1
(96351,) -1
(83766,) -1
(36180,) -1
(10891,) -1
(34236,) -1
(38032,) -1
(16667,) -1
(38556,) -1
(3168,) -1
(14175,) -1
(1325,) -1
(80372,) -1
(21,) -1
(54291,) -1
(49965,) -3
(31083,) -1
(283941,) -1
(542,) -1
(7756,) -1
(19290,) -1
(124284,) -1
(156138,) -1
(19284,) -1
(935,) -1
(48652,) -1
(50358,) -1
(286,) -1
(61624,) -2
(88363,) -1
(11698,) -1
(66751,) -1
(30,) -1
(10129,) -1
(6982,) -1
(34108,) -1
(7110,) -1
(36,) -1
(31619,) -1
(70160,) -1
(13526,) -1
(133868,) -1
(26248,) -1
(54556,) -1
(10653,) -1
(24155,) -1
(15238,) -1
(11302,) -1
(40929,) -1
(66221,) -1
(111181,) -1
(11305,) -1
(21139,) -3
(45773,) -1
(46952,) -1
(113935,) -1
(35165,) -1
(10647,) -1
(42894,) -1
(111574,) -1
(1599,) -3
(9716

In [97]:
group_df.unique("is_preparation_alert")

Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,ActionGrouped,ActionGranular,EntityType,EvidenceRole,DeviceId,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,EmailClusterId,RegistryKey,RegistryValueName,RegistryValueData,ApplicationId,ApplicationName,OAuthApplicationId,ThreatFamily,FileName,FolderPath,ResourceIdName,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City,dataset_type,is_preparation_alert,is_intrusion_alert,is_compromise_alert,alert_bit_value
i64,i64,i64,i64,datetime[μs],i64,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,str,str,i64,i64,str,str,str,i64,i64,i64,str,bool,bool,bool,i64
25769806977,415,149034,1632804,2024-06-06 18:01:00,5351,39987,"""Collection""","""T1114.002""","""TruePositive""",null,null,"""User""","""Impacted""",98799,138268,360606,160396,356591,567603,389217,389916,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",false,true,false,2


In [98]:
group_df.select(
    pl.max_horizontal(
        pl.col("is_preparation_alert").cast(int),
        pl.col("is_intrusion_alert").cast(int)*2,
        pl.col("is_compromise_alert").cast(int)*4
    ).alias("phase")
    .diff().fill_null(0).alias("phase_diff")
).get_column("phase_diff").min()

0

In [99]:
dataset.data.filter((pl.col("alert_bit_value") == 5))

Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,ActionGrouped,ActionGranular,EntityType,EvidenceRole,DeviceId,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,EmailClusterId,RegistryKey,RegistryValueName,RegistryValueData,ApplicationId,ApplicationName,OAuthApplicationId,ThreatFamily,FileName,FolderPath,ResourceIdName,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City,dataset_type,is_preparation_alert,is_intrusion_alert,is_compromise_alert,alert_bit_value
i64,i64,i64,i64,datetime[μs],i64,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,str,str,i64,i64,str,str,str,i64,i64,i64,str,bool,bool,bool,i64
1589137904239,38,550104,1651277,2024-06-05 15:08:41,6164,55756,"""Impact""","""T1496;T1585""","""BenignPositive""",null,null,"""User""","""Impacted""",98799,138268,360606,160396,423606,565616,322947,192567,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",true,false,true,5
1151051239815,78,225772,1656384,2024-06-14 15:03:45,7146,20674,"""Impact""","""T1496;T1585""","""BenignPositive""",null,null,"""User""","""Impacted""",98799,138268,360606,160396,65384,110273,75714,69404,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",true,false,true,5
1142461305190,78,225693,1640614,2024-06-11 17:01:21,7146,20674,"""Impact""","""T1496;T1585""","""BenignPositive""",null,null,"""User""","""Impacted""",98799,138268,360606,160396,65384,110273,75714,69404,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",true,false,true,5
1460288881471,107,130792,1542394,2024-06-13 18:42:06,4487,20674,"""Impact""","""T1496;T1585""","""BenignPositive""",null,null,"""User""","""Impacted""",98799,138268,360606,160396,46810,46970,48009,13277,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",true,false,true,5
841813591901,1536,51696,1714127,2024-06-12 16:38:47,8965,20674,"""Impact""","""T1496;T1585""","""BenignPositive""",null,null,"""User""","""Impacted""",98799,138268,360606,160396,357304,483782,336361,340070,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",true,false,true,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
730144441155,107,159988,1636946,2024-06-10 14:07:57,4487,20674,"""Impact""","""T1496;T1585""","""BenignPositive""",null,null,"""User""","""Impacted""",98799,138268,360606,160396,46810,46970,48009,13277,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""train""",true,false,true,5
1382979470160,107,99055,1635766,2024-06-07 12:14:30,4487,20674,"""Impact""","""T1496;T1585""","""BenignPositive""",null,null,"""Ip""","""Related""",98799,138268,46280,160396,441377,673934,425863,453297,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""test""",true,false,true,5
266287973122,107,110207,1653642,2024-06-03 02:48:40,4487,20674,"""Impact""","""T1496;T1585""","""BenignPositive""",null,null,"""User""","""Impacted""",98799,138268,360606,160396,422678,554925,401687,382299,153085,529644,null,1631,635,860,2251,3421,881,null,289573,117668,3586,null,null,5,66,null,null,null,242,1445,10630,"""test""",true,false,true,5
